In [1]:
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, classification_report
import numpy as np
import matplotlib.pylab as plt
import datetime as dt
import time

In [34]:
def load_snp_returns():
    f = open("../datasets/snp/table.csv", "r").readlines()[1:]
    raw_data = []
    raw_dates = []
    i = 0
    for line in f:
       # if i == 0 or i % 500000 == 0: 
       #     print(line)
        try:
            splits = line.split(',')
            open_price = float(splits[1])
            close_price = float(splits[4])
            raw_data.append(close_price - open_price)
            raw_dates.append(splits[0])
        except:
            continue
    
    return raw_data[::-1], raw_dates[::-1]

In [37]:
data = load_snp_returns()

In [19]:
import pandas as pd

def load_snp_returns_pd():
    return pd.read_csv("../datasets/snp/table.csv")

In [20]:
data_pd = load_snp_returns_pd()

In [21]:
data_pd.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,2016-05-23,2052.229980,2055.580078,2047.260010,2048.040039,3055480000,2048.040039
1,2016-05-20,2041.880005,2058.350098,2041.880005,2052.320068,3507650000,2052.320068
2,2016-05-19,2044.209961,2044.209961,2025.910034,2040.040039,3846770000,2040.040039
3,2016-05-18,2044.380005,2060.610107,2034.489990,2047.630005,4101320000,2047.630005
4,2016-05-17,2065.040039,2065.689941,2040.819946,2047.209961,4108960000,2047.209961


In [41]:
def load_snp_close():
    f = open('../datasets/snp/table.csv', 'r').readlines()[1:]
    raw_data = []
    raw_dates = []
    for line in f:
        try:
            splits = line.split(',')
            close_price = float(splits[4])
            raw_data.append(close_price)
            raw_dates.append(splits[0])
        except:
            continue
            
    return raw_data, raw_dates
            

In [43]:
snp_close = load_snp_close()

In [ ]:
def split_into_chunks(data, train, predict, step, binary=True, scale=True):
    X, Y = [], []
    for i in range(0, len(data), step):
        try:
            
            x_i = data[i:i+train]
            y_i = data[i+train+predict]
            
            if binary:
                if y_i > 0:
                    y_i = [1., 0.]
                else:
                    y_i = [0., 1.]
                    
                if scale: x_i = preprocessing.scale(x_i)
            
            else:
                timeseries = np.array(data[i:i+train+predict])
                if scale: timeseries = preprocessing.scale(timeseries)
                    x_i = timeseries[:-1]
                    y_i = timeseries[-1]
                    
        except:
            break
            
        X.append(x_i)
        Y.append(y_i)
        
        return X, Y
                
        

In [44]:
def shuffle_in_unison(a, b):
    # courtsey http://stackoverflow.com/users/190280/josh-bleecher-snyder
    
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    
    return shuffled_a, shuffled_b

In [45]:
def create_Xt_Yt(X, y, percentage=0.8):
    X_train = X[0:len(X) * percentage]
    Y_train = y[0:len(y) * percentage]
    
    X_train, Y_train = shuffle_in_unison(X_train, Y_train)
    
    X_test = X[len(X) * percentage]
    Y_test = y[len(X) * percentage]
    
    return X_train, X_test, Y_train, Y_test